In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping
import tensorflow as tf
import time

In [ ]:
# Wczytaj
# - model.h5
# - train.csv

In [2]:
model = tf.keras.models.load_model('model.h5')

In [3]:
datagen = ImageDataGenerator(
            featurewise_center=False,  
            samplewise_center=False,  
            featurewise_std_normalization=False,  
            samplewise_std_normalization=False,  
            zca_whitening=False, 
            rotation_range=10,  
            zoom_range = 0.1, 
            width_shift_range=0.1, 
            height_shift_range=0.1,  
            horizontal_flip=False,  
            vertical_flip=False)  

In [4]:
def create_model():
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last',
                     input_shape=(28, 28, 1)))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding='same', activation='relu', data_format='channels_last'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), padding='valid', strides=2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(1024, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    return model


def adversarial_training(x_train, y_train, model, x_test, y_test, batch_size, epochs, split, attack_type, c):

    print(f"\n### Rozpoczynam trenowanie dla splitu {split * 100}% ###\n")
    start_time = time.time()

    num_adversarial_train = int(split * len(x_train))
    print(f"Generowanie {num_adversarial_train} przykładów adwersarialnych...")

    adversarial_batch_size_train = 63  
    adversarial_images_train = []

    for i in range(0, num_adversarial_train, adversarial_batch_size_train):
      batch_images = x_train[i:i + adversarial_batch_size_train]
      batch_labels = y_train[i:i + adversarial_batch_size_train]
      print(f"Od {i} do {i + adversarial_batch_size_train} z {num_adversarial_train}")
      if attack_type == 'PGD':
        adversarial_images_train.extend(
            pgd_attack(
                model=model,
                images=batch_images,
                labels=batch_labels,
                epsilon=0.1,
                c=c,
                num_iterations=50
            )
        )
      elif attack_type == 'CW':
        adversarial_images_train.extend(cw_attack(
              model=model,
              images=batch_images,
              labels=batch_labels,
              norm="L2",
              c=c,
              kappa=0,
              max_iter=50,
              learning_rate=0.01
          ))

    print(f"Przykłady adwersarialne treningowe wygenerowane w czasie: {time.time() - start_time:.2f} s")


    num_adversarial_test = int(split * len(x_test))
    print(f"Generowanie {num_adversarial_train} przykładów adwersarialnych...")

    adversarial_batch_size_test = 42  # or even smaller, adjust as needed
    adversarial_images_test = []

    for i in range(0, num_adversarial_test, adversarial_batch_size_test):
      print(f"Od {i} do {i + adversarial_batch_size_test} z {num_adversarial_test}")
      batch_images = x_train[i:i + adversarial_batch_size_test]
      batch_labels = y_train[i:i + adversarial_batch_size_test]
      if attack_type == 'PGD':
        adversarial_images_test.extend(
            pgd_attack(
                model=model,
                images=batch_images,
                labels=batch_labels,
                norm_type='L2',
                epsilon=0.1,
                c=c,
                num_iterations=50
            )
        )
      elif attack_type == 'CW':
        adversarial_images_test.extend(cw_attack(
              model=model,
              images=batch_images,
              labels=batch_labels,
              norm="L2",
              c=c,
              kappa=0,
              max_iter=50,
              learning_rate=0.01
          ))

    adversarial_images_train = np.array(adversarial_images_train)
    adversarial_labels_train = y_train[:num_adversarial_train]

    adversarial_images_test = np.array(adversarial_images_test)
    adversarial_labels_test = y_test[:num_adversarial_test]

    # Combine Original and Adversarial Data
    print("Łączenie danych oryginalnych i adwersarialnych...")
    x_train_combined = np.concatenate([x_train, adversarial_images_train])
    y_train_combined = np.concatenate([y_train, adversarial_labels_train])

    x_test_combined = np.concatenate([x_test, adversarial_images_test])
    y_test_combined = np.concatenate([y_test, adversarial_labels_test])

    print(f"x_train_combined shape: {x_train_combined.shape}")
    print(f"y_train_combined shape: {y_train_combined.shape}")

    print(f"x_test_combined shape: {x_test_combined.shape}")
    print(f"y_test_combined shape: {y_test_combined.shape}")

    # Create and Compile a New Model
    new_model = create_model()
    optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
    new_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

    # Learning Rate Scheduler
    reduce_lr = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)

    # Early Stopping
    early_stopping = EarlyStopping(
        min_delta=0.001,
        patience=20,
        restore_best_weights=True
    )

    print("Rozpoczęcie trenowania modelu...")
    # Trening modelu pomocniczego
    history = new_model.fit(
        datagen.flow(x_train_combined, y_train_combined, batch_size=64),
        epochs=50,
        validation_data=(x_test_combined, y_test_combined),
        verbose=1,
        steps_per_epoch=x_train_combined.shape[0] // 64,
        callbacks=[reduce_lr, early_stopping]
    )

    elapsed_time = time.time() - start_time
    print(f"Model wytrenowany dla splitu {split * 100}% w czasie: {elapsed_time:.2f} s\n")

    return new_model

In [5]:
import tensorflow as tf
import numpy as np
import statistics as stat

def cw_attack(model, images, labels, norm='L2', c=1e-4, kappa=0, max_iter=1000, learning_rate=0.01):
    """
    Atak CW z obsługą norm L2, Linf.
    Argumenty:
        model: Model Keras, na który przeprowadzany jest atak.
        images: Obrazy wejściowe, przeskalowane do zakresu [0, 1].
        labels: Prawdziwe etykiety dla ataku nietargetowanego lub etykiety docelowe dla ataku targetowanego (zakodowane w formie one-hot).
        norm: Typ normy dla perturbacji ('L2', 'Linf').
        c: Parametr regularyzacyjny kontrolujący rozmiar perturbacji.
        kappa: Margines pewności ataku.
        max_iter: Maksymalna liczba kroków optymalizacji.
        learning_rate: Współczynnik uczenia dla optymalizatora.
    Zwraca:
        Zmienione obrazy zawierające atak adversarialny.
    """

    # Konwersja obrazów i etykiet na tensory
    original_images = tf.convert_to_tensor(images, dtype=tf.float32)
    true_labels = tf.convert_to_tensor(labels, dtype=tf.float32)

    # Inicjalizacja perturbacji w przestrzeni tanh
    w = tf.Variable(tf.zeros_like(original_images), trainable=True, dtype=tf.float32)

    # Optymalizator
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    # Definicja funkcji f
    def f(x):
        logits = model(x)
        one_hot_labels = true_labels
        true_logits = tf.reduce_sum(one_hot_labels * logits, axis=1)
        other_logits = tf.reduce_max((1 - one_hot_labels) * logits - one_hot_labels * 1e4, axis=1)
        return tf.maximum(true_logits - other_logits, -kappa)

    # Definicja funkcji normy perturbacji
    def perturbation_norm(perturbed_images, original_images, norm):
        if norm == 'L2':
            return tf.reduce_sum(tf.square(perturbed_images - original_images), axis=[1, 2, 3])
        elif norm == 'Linf':
            return tf.reduce_max(tf.abs(perturbed_images - original_images), axis=[1, 2, 3])
        else:
            raise ValueError(f"Nieobsługiwana norma: {norm}")

    # Wykonanie optymalizacji
    prev_loss = 1e10
    for step in range(max_iter):
        with tf.GradientTape() as tape:
            # Mapowanie w z powrotem do przestrzeni obrazów przy użyciu funkcji tanh
            perturbed_images = 0.5 * (tf.tanh(w) + 1)

            # Obliczanie składowych funkcji straty
            norm_loss = perturbation_norm(perturbed_images, original_images, norm)
            f_loss = c * f(perturbed_images)
            total_loss = tf.reduce_sum(norm_loss + f_loss)

        # Obliczanie gradientów i aktualizacja wartości
        gradients = tape.gradient(total_loss, [w])
        optimizer.apply_gradients(zip(gradients, [w]))

        # Sprawdzanie warunku wcześniejszego zatrzymania
        if step % (max_iter // 10) == 0:
            #print(f"C:{c} Krok {step}/{max_iter}, Strata: {total_loss.numpy()}, norm_loss: {stat.mean(norm_loss.numpy())}, f_loss: {stat.mean(f_loss.numpy())}")
            if total_loss.numpy() > prev_loss:
                print("Atak zatrzymany z powodu konwergencji.")
                break
            prev_loss = total_loss.numpy()

    # Mapowanie końcowego w z powrotem do przestrzeni obrazów
    attack_images = 0.5 * (tf.tanh(w) + 1)
    return attack_images


In [6]:
def pgd_attack(model, images, labels, epsilon, c, num_iterations, norm_type='L2', threshold=None):
    """
    Przeprowadza atak PGD z kontrolą normy perturbacji i możliwością wcześniejszego zakończenia.

    Parametry:
    - model: Model Keras lub TensorFlow
    - images: Obrazy wejściowe (batch)
    - labels: Etykiety one-hot dla obrazów
    - epsilon: Maksymalna wielkość perturbacji
    - alpha: Krok każdej iteracji
    - num_iterations: Maksymalna liczba iteracji
    - norm_type: Typ normy ( 'l2', 'linf')
    - threshold: Próg zmiany perturbacji dla wcześniejszego zakończenia (opcjonalny)

    Zwraca:
    - Adwersarialne obrazy
    """
    # Inicjalizujemy perturbacje jako kopię obrazów wejściowych
    perturbed_images = tf.identity(images)

    for i in range(num_iterations):
        with tf.GradientTape() as tape:
            tape.watch(perturbed_images)
            predictions = model(perturbed_images)
            loss = tf.keras.losses.categorical_crossentropy(labels, predictions)

        # Obliczamy gradient
        gradient = tape.gradient(loss, perturbed_images)

        # Generowanie perturbacji na podstawie wybranej normy
        if norm_type == 'Linf':
            perturbation = c * tf.sign(gradient)
        elif norm_type == 'L2':
            # Ręczne obliczenie normy L2
            norm = tf.sqrt(tf.reduce_sum(tf.square(gradient), axis=(1, 2, 3), keepdims=True))
            perturbation = c * gradient / (norm + 1e-10)
        else:
            raise ValueError("Nieobsługiwana norma. Użyj 'l2' lub 'linf'.")

        # Aktualizacja obrazów adwersarialnych
        new_perturbed_images = perturbed_images + perturbation
        new_perturbed_images = tf.clip_by_value(new_perturbed_images, images - epsilon, images + epsilon)  # projekcja
        new_perturbed_images = tf.clip_by_value(new_perturbed_images, 0, 1)  # ograniczenie do zakresu [0,1]

        # Sprawdzenie progu zmiany perturbacji
        if threshold is not None:
            delta = tf.reduce_max(tf.abs(new_perturbed_images - perturbed_images))
            if delta < threshold:
                print(f"Zatrzymanie po {i + 1} iteracjach (zmiana < {threshold})")
                break

        perturbed_images = new_perturbed_images

    return perturbed_images


In [7]:
data = pd.read_csv('train.csv')

# X is the pixels and Y is the image labels
X = data.iloc[:,1:]
Y = data.iloc[:,0]
#first param in reshape is number of examples. We can pass -1 here as we want numpy to figure that out by itself

#reshape(examples, height, width, channels)
X_reshaped = X.values.reshape(-1, 28, 28, 1)

In [8]:
#splitting dataframe using train_test_split
x_train , x_test , y_train , y_test = train_test_split(X_reshaped, Y, test_size=0.1)

#convert values to float as result will be a float. If not done vals are set to zero
x_train = x_train.astype("float32")/255
x_test = x_test.astype("float32")/255

#fitting the ImageDataGenerator
datagen.fit(x_train)

In [9]:
#notice num_classes is set to 10 as we have 10 different labels
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [10]:
# Hyperparameters
batch_size = 64
epochs = 50
splits = [0.2, 0.3, 0.4]

In [13]:
# Train and Save Models
for split in splits:
    start_time = time.time()
    print(f"\nTrening modelu dla splitu {split * 100}% danych adwersarialnych...")

    # Trenowanie modelu
    model = adversarial_training(x_train, y_train, model, x_test, y_test, batch_size, epochs, split=split, attack_type = "PGD", c = 0.3)

    # Nazwa pliku
    model_filename = f"AD_model_PGD_{split:.1f}.h5"

    # Zapis modelu w bieżącym katalogu
    model.save(model_filename)
    print(f"Model dla splitu {split * 100}% zapisany jako {model_filename}")

    end_time = time.time()
    iteration_time = end_time - start_time
    print(f"Czas trwania treningu dla splitu {split * 100}%: {iteration_time:.2f} sekund")

print("\nWszystkie modele zostały wytrenowane i zapisane.")



Trening modelu dla splitu 20.0% danych adwersarialnych...

### Rozpoczynam trenowanie dla splitu 20.0% ###

Generowanie 7560 przykładów adwersarialnych...
Od 0 do 63 z 7560
Od 63 do 126 z 7560
Od 126 do 189 z 7560
Od 189 do 252 z 7560
Od 252 do 315 z 7560
Od 315 do 378 z 7560
Od 378 do 441 z 7560
Od 441 do 504 z 7560
Od 504 do 567 z 7560
Od 567 do 630 z 7560
Od 630 do 693 z 7560
Od 693 do 756 z 7560
Od 756 do 819 z 7560
Od 819 do 882 z 7560
Od 882 do 945 z 7560
Od 945 do 1008 z 7560
Od 1008 do 1071 z 7560
Od 1071 do 1134 z 7560
Od 1134 do 1197 z 7560
Od 1197 do 1260 z 7560
Od 1260 do 1323 z 7560
Od 1323 do 1386 z 7560
Od 1386 do 1449 z 7560
Od 1449 do 1512 z 7560
Od 1512 do 1575 z 7560
Od 1575 do 1638 z 7560
Od 1638 do 1701 z 7560
Od 1701 do 1764 z 7560
Od 1764 do 1827 z 7560
Od 1827 do 1890 z 7560
Od 1890 do 1953 z 7560
Od 1953 do 2016 z 7560
Od 2016 do 2079 z 7560
Od 2079 do 2142 z 7560
Od 2142 do 2205 z 7560
Od 2205 do 2268 z 7560
Od 2268 do 2331 z 7560
Od 2331 do 2394 z 7560
Od 23

Model wytrenowany dla splitu 20.0% w czasie: 532.16 s

Model dla splitu 20.0% zapisany jako AD_model_PGD_0.2.h5
Czas trwania treningu dla splitu 20.0%: 532.26 sekund

Trening modelu dla splitu 30.0% danych adwersarialnych...

### Rozpoczynam trenowanie dla splitu 30.0% ###

Generowanie 11340 przykładów adwersarialnych...
Od 0 do 63 z 11340
Od 63 do 126 z 11340
Od 126 do 189 z 11340
Od 189 do 252 z 11340
Od 252 do 315 z 11340
Od 315 do 378 z 11340
Od 378 do 441 z 11340
Od 441 do 504 z 11340
Od 504 do 567 z 11340
Od 567 do 630 z 11340
Od 630 do 693 z 11340
Od 693 do 756 z 11340
Od 756 do 819 z 11340
Od 819 do 882 z 11340
Od 882 do 945 z 11340
Od 945 do 1008 z 11340
Od 1008 do 1071 z 11340
Od 1071 do 1134 z 11340
Od 1134 do 1197 z 11340
Od 1197 do 1260 z 11340
Od 1260 do 1323 z 11340
Od 1323 do 1386 z 11340
Od 1386 do 1449 z 11340
Od 1449 do 1512 z 11340
Od 1512 do 1575 z 11340
Od 1575 do 1638 z 11340
Od 1638 do 1701 z 11340
Od 1701 do 1764 z 11340
Od 1764 do 1827 z 11340
Od 1827 do 1890 

Model wytrenowany dla splitu 30.0% w czasie: 829.35 s

Model dla splitu 30.0% zapisany jako AD_model_PGD_0.3.h5
Czas trwania treningu dla splitu 30.0%: 829.45 sekund

Trening modelu dla splitu 40.0% danych adwersarialnych...

### Rozpoczynam trenowanie dla splitu 40.0% ###

Generowanie 15120 przykładów adwersarialnych...
Od 0 do 63 z 15120
Od 63 do 126 z 15120
Od 126 do 189 z 15120
Od 189 do 252 z 15120
Od 252 do 315 z 15120
Od 315 do 378 z 15120
Od 378 do 441 z 15120
Od 441 do 504 z 15120
Od 504 do 567 z 15120
Od 567 do 630 z 15120
Od 630 do 693 z 15120
Od 693 do 756 z 15120
Od 756 do 819 z 15120
Od 819 do 882 z 15120
Od 882 do 945 z 15120
Od 945 do 1008 z 15120
Od 1008 do 1071 z 15120
Od 1071 do 1134 z 15120
Od 1134 do 1197 z 15120
Od 1197 do 1260 z 15120
Od 1260 do 1323 z 15120
Od 1323 do 1386 z 15120
Od 1386 do 1449 z 15120
Od 1449 do 1512 z 15120
Od 1512 do 1575 z 15120
Od 1575 do 1638 z 15120
Od 1638 do 1701 z 15120
Od 1701 do 1764 z 15120
Od 1764 do 1827 z 15120
Od 1827 do 1890 

Model wytrenowany dla splitu 40.0% w czasie: 951.10 s

Model dla splitu 40.0% zapisany jako AD_model_PGD_0.4.h5
Czas trwania treningu dla splitu 40.0%: 951.21 sekund

Wszystkie modele zostały wytrenowane i zapisane.


In [11]:
# Train and Save Models
for split in splits:
    start_time = time.time()
    print(f"\nTrening modelu dla splitu {split * 100}% danych adwersarialnych...")

    # Trenowanie modelu
    model = adversarial_training(x_train, y_train, model, x_test, y_test, batch_size, epochs, split=split, attack_type = "CW", c = 0.3)

    # Nazwa pliku
    model_filename = f"AD_model_CW_{split:.1f}.h5"

    # Zapis modelu w bieżącym katalogu
    model.save(model_filename)
    print(f"Model dla splitu {split * 100}% zapisany jako {model_filename}")

    end_time = time.time()
    iteration_time = end_time - start_time
    print(f"Czas trwania treningu dla splitu {split * 100}%: {iteration_time:.2f} sekund")

print("\nWszystkie modele zostały wytrenowane i zapisane.")


Trening modelu dla splitu 20.0% danych adwersarialnych...

### Rozpoczynam trenowanie dla splitu 20.0% ###

Generowanie 7560 przykładów adwersarialnych...
Od 0 do 63 z 7560
Od 63 do 126 z 7560
Od 126 do 189 z 7560
Od 189 do 252 z 7560
Od 252 do 315 z 7560
Od 315 do 378 z 7560
Od 378 do 441 z 7560
Od 441 do 504 z 7560
Od 504 do 567 z 7560
Od 567 do 630 z 7560
Od 630 do 693 z 7560
Od 693 do 756 z 7560
Od 756 do 819 z 7560
Od 819 do 882 z 7560
Od 882 do 945 z 7560
Od 945 do 1008 z 7560
Od 1008 do 1071 z 7560
Od 1071 do 1134 z 7560
Od 1134 do 1197 z 7560
Od 1197 do 1260 z 7560
Od 1260 do 1323 z 7560
Od 1323 do 1386 z 7560
Od 1386 do 1449 z 7560
Od 1449 do 1512 z 7560
Od 1512 do 1575 z 7560
Od 1575 do 1638 z 7560
Od 1638 do 1701 z 7560
Od 1701 do 1764 z 7560
Od 1764 do 1827 z 7560
Od 1827 do 1890 z 7560
Od 1890 do 1953 z 7560
Od 1953 do 2016 z 7560
Od 2016 do 2079 z 7560
Od 2079 do 2142 z 7560
Od 2142 do 2205 z 7560
Od 2205 do 2268 z 7560
Od 2268 do 2331 z 7560
Od 2331 do 2394 z 7560
Od 23

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Rozpoczęcie trenowania modelu...
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


708/708 ━━━━━━━━━━━━━━━━━━━━ 36s 37ms/step - accuracy: 0.7832 - loss: 0.7707 - val_accuracy: 0.8331 - val_loss: 1.9411 - learning_rate: 0.0010
Epoch 2/50
  1/708 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.9375 - loss: 0.1409

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


708/708 ━━━━━━━━━━━━━━━━━━━━ 0s 474us/step - accuracy: 0.9375 - loss: 0.1409 - val_accuracy: 0.8335 - val_loss: 1.9514 - learning_rate: 9.0000e-04
Epoch 3/50
708/708 ━━━━━━━━━━━━━━━━━━━━ 19s 26ms/step - accuracy: 0.9586 - loss: 0.1338 - val_accuracy: 0.8415 - val_loss: 2.0438 - learning_rate: 8.1000e-04
Epoch 4/50
708/708 ━━━━━━━━━━━━━━━━━━━━ 1s 940us/step - accuracy: 0.9531 - loss: 0.1611 - val_accuracy: 0.8419 - val_loss: 2.0302 - learning_rate: 7.2900e-04
Epoch 5/50
708/708 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - accuracy: 0.9700 - loss: 0.0959 - val_accuracy: 0.8440 - val_loss: 2.1601 - learning_rate: 6.5610e-04
Epoch 6/50
708/708 ━━━━━━━━━━━━━━━━━━━━ 0s 506us/step - accuracy: 0.9375 - loss: 0.2546 - val_accuracy: 0.8444 - val_loss: 2.1576 - learning_rate: 5.9049e-04
Epoch 7/50
708/708 ━━━━━━━━━━━━━━━━━━━━ 23s 30ms/step - accuracy: 0.9763 - loss: 0.0746 - val_accuracy: 0.8444 - val_loss: 2.0275 - learning_rate: 5.3144e-04
Epoch 8/50
708/708 ━━━━━━━━━━━━━━━━━━━━ 1s 917us/step - accurac

Model wytrenowany dla splitu 20.0% w czasie: 600.22 s

Model dla splitu 20.0% zapisany jako AD_model_CW_0.2.h5
Czas trwania treningu dla splitu 20.0%: 600.33 sekund

Trening modelu dla splitu 30.0% danych adwersarialnych...

### Rozpoczynam trenowanie dla splitu 30.0% ###

Generowanie 11340 przykładów adwersarialnych...
Od 0 do 63 z 11340
Od 63 do 126 z 11340
Od 126 do 189 z 11340
Od 189 do 252 z 11340
Od 252 do 315 z 11340
Od 315 do 378 z 11340
Od 378 do 441 z 11340
Od 441 do 504 z 11340
Od 504 do 567 z 11340
Od 567 do 630 z 11340
Od 630 do 693 z 11340
Od 693 do 756 z 11340
Od 756 do 819 z 11340
Od 819 do 882 z 11340
Od 882 do 945 z 11340
Od 945 do 1008 z 11340
Od 1008 do 1071 z 11340
Od 1071 do 1134 z 11340
Od 1134 do 1197 z 11340
Od 1197 do 1260 z 11340
Od 1260 do 1323 z 11340
Od 1323 do 1386 z 11340
Od 1386 do 1449 z 11340
Od 1449 do 1512 z 11340
Od 1512 do 1575 z 11340
Od 1575 do 1638 z 11340
Od 1638 do 1701 z 11340
Od 1701 do 1764 z 11340
Od 1764 do 1827 z 11340
Od 1827 do 1890 z

Model wytrenowany dla splitu 30.0% w czasie: 792.85 s

Model dla splitu 30.0% zapisany jako AD_model_CW_0.3.h5
Czas trwania treningu dla splitu 30.0%: 792.95 sekund

Trening modelu dla splitu 40.0% danych adwersarialnych...

### Rozpoczynam trenowanie dla splitu 40.0% ###

Generowanie 15120 przykładów adwersarialnych...
Od 0 do 63 z 15120
Od 63 do 126 z 15120
Od 126 do 189 z 15120
Od 189 do 252 z 15120
Od 252 do 315 z 15120
Od 315 do 378 z 15120
Od 378 do 441 z 15120
Od 441 do 504 z 15120
Od 504 do 567 z 15120
Od 567 do 630 z 15120
Od 630 do 693 z 15120
Od 693 do 756 z 15120
Od 756 do 819 z 15120
Od 819 do 882 z 15120
Od 882 do 945 z 15120
Od 945 do 1008 z 15120
Od 1008 do 1071 z 15120
Od 1071 do 1134 z 15120
Od 1134 do 1197 z 15120
Od 1197 do 1260 z 15120
Od 1260 do 1323 z 15120
Od 1323 do 1386 z 15120
Od 1386 do 1449 z 15120
Od 1449 do 1512 z 15120
Od 1512 do 1575 z 15120
Od 1575 do 1638 z 15120
Od 1638 do 1701 z 15120
Od 1701 do 1764 z 15120
Od 1764 do 1827 z 15120
Od 1827 do 1890 z

Model wytrenowany dla splitu 40.0% w czasie: 950.77 s

Model dla splitu 40.0% zapisany jako AD_model_CW_0.4.h5
Czas trwania treningu dla splitu 40.0%: 950.90 sekund

Wszystkie modele zostały wytrenowane i zapisane.
